In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import time
from tqdm import tqdm  # 导入tqdm库

# 1. 加载数据
data = pd.read_csv('./data/total_load_actual.csv')

# 2. 检查数据是否包含 NaN
if data.isnull().any().any():
    print("Data contains NaN values. Filling missing values...")
    data = data.ffill()  # 使用前值填充缺失数据
else:
    print("Data does not contain NaN values.")

# 3. 将数据列转换为字典格式
data_dict = {}
for col in data.columns:
    data_dict[col] = data[col].tolist()

total_acual_load_list = data_dict['total load actual']

# 4. 归一化数据
scaler = MinMaxScaler(feature_range=(0, 1))
total_acual_load_list_scaled = scaler.fit_transform(np.array(total_acual_load_list).reshape(-1, 1)).flatten()

# 5. 设定序列长度和样本数
seq_length = 24
num_samples = len(total_acual_load_list_scaled) - seq_length
# 6. 初始化输入和输出数据
x_processed = []
y_processed = []

window_size = 8
i = 0
while i * window_size + seq_length-1 < len(total_acual_load_list_scaled):
    x_processed.append(total_acual_load_list_scaled[i * window_size:i * window_size + seq_length-2])
    y_processed.append(total_acual_load_list_scaled[i * window_size + seq_length-1])
    i += 1

# 7. 转换为NumPy数组
x_processed = np.array(x_processed)
y_processed = np.array(y_processed)

# 8. 转换为Tensor
x_processed_tensor = torch.tensor(x_processed, dtype=torch.float32)
y_processed_tensor = torch.tensor(y_processed, dtype=torch.float32).reshape(-1, 1)

# 9. 数据形状检查: 确保 x_processed_tensor 的形状为 (样本数量, 序列长度-1, 输入维度)
x_processed_tensor = x_processed_tensor.unsqueeze(-1)  # 将输入扩展为三维张量 (样本数量, seq_length-1, input_dim)

# 10. 数据划分: 80% 训练集,20% 测试集 使用 train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_processed_tensor, y_processed_tensor, test_size=0.2, random_state=42)

# 11. 定义改进的堆叠GRU-RNN模型
class ImprovedGRUCell(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(ImprovedGRUCell, self).__init__()
        self.hidden_dim = hidden_dim
        
        # 更新门和重置门的权重和偏置
        self.update_gate = nn.Linear(hidden_dim, hidden_dim, bias=True)
        self.reset_gate = nn.Linear(hidden_dim, hidden_dim, bias=True)
        
        # 候选层的权重和偏置
        self.candidate_layer = nn.Linear(input_dim + hidden_dim, hidden_dim, bias=True)
        
    def forward(self, x, h):
        # 计算更新门
        z = torch.sigmoid(self.update_gate(h))
        
        # 计算重置门
        r = torch.sigmoid(self.reset_gate(h))
        
        # 计算候选层
        h_tilde = torch.tanh(self.candidate_layer(torch.cat([x, r * h], dim=1)))
        
        # 更新隐藏状态
        h_new = (1 - z) * h + z * h_tilde
        
        return h_new

class StackedGRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=2):
        super(StackedGRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # 使用改进的GRU单元
        self.gru_cells = nn.ModuleList([ImprovedGRUCell(input_dim, hidden_dim)])
        for _ in range(1, num_layers):
            self.gru_cells.append(ImprovedGRUCell(hidden_dim, hidden_dim))
        
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h = [torch.zeros(x.size(0), self.hidden_dim).to(x.device) for _ in range(self.num_layers)]
        
        for t in range(x.size(1)):
            for layer in range(self.num_layers):
                if layer == 0:
                    h[layer] = self.gru_cells[layer](x[:, t, :], h[layer])
                else:
                    h[layer] = self.gru_cells[layer](h[layer-1], h[layer])
        
        out = self.fc(h[-1])
        return out

# 12. 定义模型
input_dim = 1  # 输入维度
hidden_dim = 32
output_dim = 1
num_layers = 1
model = StackedGRU(input_dim, hidden_dim, output_dim, num_layers)

# 13.检查是否有可用的GPU,如果有则使用GPU,否则使用CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 将模型移动到设备上
model.to(device)

# 14. 定义损失函数和优化器
criterion = nn.MSELoss()

# 使用adams
optimizer = optim.Adam(
    model.parameters(), 
    lr=0.04
)# 15. 加载权重文件(如果存在)
weight_file = './weights/cnn_lstm_model.pth'
hyperparams_file = './weights/hyperparams.txt'

# 如果有预训练的权重文件,则加载
if os.path.exists(weight_file):
    print(f"Loading pre-trained model from {weight_file}")
    model.load_state_dict(torch.load(weight_file))
else:
    print("No pre-trained model found, starting from scratch.")


# 14. 训练模型
num_epochs = 1000
train_loss = []
print('Training model...')
start_time = time.time()  # 记录训练开始时间

# 使用tqdm显示进度条,并设置更新频率
for epoch in tqdm(range(num_epochs), desc="Training", unit="epoch", mininterval=1, maxinterval=10):
    model.train()
    optimizer.zero_grad()
    
    # 将数据移动到设备上
    x_train_device = x_train.to(device)
    y_train_device = y_train.to(device)
    
    output = model(x_train_device)
    loss = criterion(output, y_train_device)
    loss.backward()
    optimizer.step()
    
    train_loss.append(loss.item())
    if (epoch+1) % (num_epochs//10) == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

end_time = time.time()  # 记录训练结束时间
training_time = end_time - start_time  # 计算训练耗时
print(f'Training completed in {training_time:.2f} seconds')

Data contains NaN values. Filling missing values...
Using device: cpu
Training model...


Training:  10%|▉         | 97/1000 [00:16<02:35,  5.79epoch/s]

Epoch [100/1000], Loss: 0.0082


Training:  20%|█▉        | 196/1000 [00:34<02:34,  5.20epoch/s]

Epoch [200/1000], Loss: 0.0032


Training:  20%|██        | 202/1000 [00:35<02:45,  4.83epoch/s]

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

# 计算各类指标的函数
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = np.mean(np.abs(y_true - y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    rmse = np.sqrt(mse)
    r2 = 1 - (np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2))
    smape = np.mean(2 * np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))
    
    return mse, mae, mape, rmse, r2, smape

# 15. 测试模型
model.eval()
with torch.no_grad():
    predicted = model(x_test).cpu().numpy()

# 16. 反归一化预测结果
predicted_original_scale = scaler.inverse_transform(predicted)
y_test_original_scale = scaler.inverse_transform(y_test.cpu().numpy())

# 17. 计算归一化前的指标
mse_original, mae_original, mape_original, rmse_original, r2_original, smape_original = calculate_metrics(y_test_original_scale, predicted_original_scale)

# 18. 计算归一化后的指标
y_test_np = y_test.cpu().numpy().flatten()
predicted_np = predicted.flatten()
mse_normalized, mae_normalized, mape_normalized, rmse_normalized, r2_normalized, smape_normalized = calculate_metrics(y_test_np, predicted_np)

# 19. 打印结果
print('+=============+=============+=============+=============+')
print('|   Metrics (Original Scale)   |   Value   |   Metrics (Normalized Scale)   |   Value   |')
print('+=============+=============+=============+=============+')
metrics = [
    ('MSE', mse_original, mse_normalized),
    ('MAE', mae_original, mae_normalized),
    ('MAPE', mape_original, mape_normalized),
    ('RMSE', rmse_original, rmse_normalized),
    ('R²', r2_original, r2_normalized),
    ('SMAPE', smape_original, smape_normalized)
]

for metric, original_value, normalized_value in metrics:
    print(f'| {metric:<15} | {original_value:>26.6f} | {metric} (Normalized){"":<2} | {normalized_value:>26.6f} |')

print('+=============+=============+=============+=============+')


+=============+=============+=============+=============+
|   Metrics (Original Scale)   |   Value   |   Metrics (Normalized Scale)   |   Value   |
+=============+=============+=============+=============+
| MSE             |             1001287.250000 | MSE (Normalized)   |                   0.001897 |
| MAE             |                 574.350037 | MAE (Normalized)   |                   0.025000 |
| MAPE            |                   2.117244 | MAPE (Normalized)   |                   7.426083 |
| RMSE            |                1000.643433 | RMSE (Normalized)   |                   0.043555 |
| R²              |                   0.927790 | R² (Normalized)   |                   0.927790 |
| SMAPE           |                   0.021032 | SMAPE (Normalized)   |                   0.070274 |
+=============+=============+=============+=============+


In [ ]:
import plotly.graph_objs as go
import plotly.offline as pyo

# 1. 训练损失的交互式可视化
train_loss_fig = go.Figure()

# 添加训练损失曲线
train_loss_fig.add_trace(go.Scatter(
    x=list(range(1, num_epochs + 1)),  # 横坐标是训练的轮数
    y=train_loss,  # 纵坐标是训练过程中的损失
    mode='lines',  # 线形显示
    name='Training Loss',  # 曲线的标签
    line=dict(color='royalblue', width=3, dash='dash')  # 设置颜色、宽度和虚线样式
))

# 更新图表的布局
train_loss_fig.update_layout(
    title='AdaGRU-RNN:Training Loss Curve',  # 图表标题
    title_font=dict(size=20, family='Times New Roman', color='black'),  # 设置标题字体
    xaxis=dict(title='Epoch', title_font=dict(size=14, family='Times New Roman', color='black'), gridcolor='lightgray'),  # X轴标签
    yaxis=dict(title='Loss', title_font=dict(size=14, family='Times New Roman', color='black'), gridcolor='lightgray'),  # Y轴标签
    plot_bgcolor='whitesmoke',  # 设置图表背景色
    paper_bgcolor='whitesmoke',  # 设置图表外部背景色
    font=dict(family='Times New Roman', size=12, color='black'),  # 字体设置
    legend=dict(
        x=0.5, y=1.05,  # 设置图例位置
        xanchor='center',  # 图例水平居中
        yanchor='bottom',  # 图例垂直居上
        traceorder='normal',  # 图例顺序
        orientation='h',  # 图例显示为水平排列(即一行多列)
        font=dict(family='Times New Roman', size=12, color='black'),
        bgcolor='rgba(255, 255, 255, 0.5)',  # 图例背景颜色
        bordercolor='black', borderwidth=1  # 图例边框颜色和宽度
    ),
    showlegend=True,  # 显示图例
    width=1200,  # 设置图表宽度
    height=600  # 设置图表高度
)

# 设置图表边缘的阴影效果
train_loss_fig.update_layout(
    xaxis=dict(showgrid=True, zeroline=False, showline=True, linecolor='gray'),  # X轴样式
    yaxis=dict(showgrid=True, zeroline=False, showline=True, linecolor='gray'),  # Y轴样式
    margin=dict(l=50, r=50, t=50, b=50)  # 设置图表的边距
)

train_loss_fig.show()

# 2. 预测与实际结果的交互式可视化
# 创建实际结果的曲线
timestep = 100
actual_trace = go.Scatter(
    x=list(range(timestep)),  # 横坐标是时间步
    y=y_test_original_scale[:timestep].flatten(),  # 纵坐标是实际测试数据(前100个数据点)
    mode='lines+markers',  # 线条和数据点显示
    name='Actual',  # 曲线名称
    line=dict(color='mediumseagreen', width=3, shape='spline'),  # 设置颜色、宽度和曲线类型
    marker=dict(symbol='circle', size=8, color='white', line=dict(color='mediumseagreen', width=2))  # 数据点样式(空心)
)

# 创建预测结果的曲线
predicted_trace = go.Scatter(
    x=list(range(timestep)),  # 横坐标是时间步
    y=predicted_original_scale[:timestep].flatten(),  # 纵坐标是模型的预测数据(前100个数据点)
    mode='lines+markers',  # 线条和数据点显示
    name='Predicted',  # 曲线名称
    line=dict(color='indianred', width=3, dash='dot'),  # 设置颜色、宽度和虚线样式
    marker=dict(symbol='x', size=8, color='white', line=dict(color='indianred', width=2))  # 数据点样式(空心)
)

# 创建包含实际和预测数据的图表
comparison_fig = go.Figure(data=[actual_trace, predicted_trace])

# 更新布局设置
comparison_fig.update_layout(
    title='AdaGRU-RNN:Actual vs Predicted',  # 图表标题
    title_font=dict(size=20, family='Times New Roman', color='black'),  # 标题字体
    xaxis=dict(title='Time Step', title_font=dict(size=14, family='Times New Roman', color='black'), gridcolor='lightgray'),  # X轴标签
    yaxis=dict(title='Load', title_font=dict(size=14, family='Times New Roman', color='black'), gridcolor='lightgray'),  # Y轴标签
    plot_bgcolor='whitesmoke',  # 图表背景颜色
    paper_bgcolor='whitesmoke',  # 外部背景颜色
    font=dict(family='Times New Roman', size=12, color='black'),  # 字体设置
    legend=dict(
        x=0.5, y=1.05,  # 设置图例位置
        xanchor='center',  # 图例水平居中
        yanchor='bottom',  # 图例垂直居上
        traceorder='normal',  # 图例顺序
        orientation='h',  # 图例显示为水平排列(即一行多列)
        font=dict(family='Times New Roman', size=12, color='black'),
        bgcolor='rgba(255, 255, 255, 0.5)',  # 图例背景颜色
        bordercolor='black', borderwidth=1  # 图例边框颜色和宽度
    ),
    showlegend=True,  # 显示图例
    width=1200,  # 设置图表宽度
    height=600  # 设置图表高度
)

# 为图表添加渐变背景色
comparison_fig.update_layout(
    paper_bgcolor='rgba(240, 240, 240, 0.9)',  # 外部背景渐变色
    plot_bgcolor='rgba(255, 255, 255, 0.9)',  # 图表区域背景渐变色
)

# 显示预测与实际结果的图表
comparison_fig.show()

: 